In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
import tensorflow as tf
import load_data, helper
import matplotlib.pyplot as plt
%matplotlib inline
from six.moves import cPickle
from tfomics import neuralnetwork as nn
from tfomics import utils, metrics, saliency, visualize

In [49]:
correspondence_path = '../correspondences_eCLIP_RNACompete.csv'
import pandas as pd

df = pd.read_csv(correspondence_path, header=None)
rbp_names = df[2].as_matrix()
rbp_indices = df[3].as_matrix()
rncmpt_names = df[1].as_matrix()


In [50]:
experiment_names = []
for key in dataset.keys():
    experiment_names.append(str(key))

In [65]:

match = {}
for j, rbp_name in enumerate(rbp_names):
    match[rbp_name] = []

for j, rbp_name in enumerate(rbp_names):
    for i, name in enumerate(experiment_names):
        if rbp_name in name:
            match[rbp_name].append((rbp_indices[j], experiment_names[i], rncmpt_names[j]))

In [72]:
clean_match = {}
for key in match.keys():
    if match[key]:
        clean_match[key] = match[key]

In [15]:
rbp_index = 69
from models import deep_residual_model as genome_model
#from models import conv_net as genome_model
normalize_method = 'log_norm' 
ss_type = 'seq'

data_path = '../data/RNAcompete_2013/rnac_SetAB_with_nan_RNAplfoldstructures.hdf5'
train_all, valid_all, test_all = load_data.RNAcompete_dataset(data_path, split_ratio=0.9, ss_type=ss_type)    

# process data 
train, valid, test = helper.process_data(train_all, valid_all, test_all, rbp_index, method=normalize_method)

In [16]:

input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = train['targets'].shape
model_layers, optimization = genome_model.model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization)

# compile neural trainer
best_path = '/home/peter/harvard/rbp/results/RNAcompete_2013/'+normalize_method+'_'+ss_type
file_path = os.path.join(best_path, str(rbp_index)+'_deep_residual_model_'+ss_type)
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)


In [17]:
# initialize session
sess = utils.initialize_session(nnmodel.placeholders)

In [18]:
# load best parameters
nntrainer.set_best_parameters(sess)

loss, mean, std = nntrainer.test_model(sess, test)

loading model from:  /home/peter/harvard/rbp/results/RNAcompete_2013/log_norm_seq/69_deep_residual_model_seq_best.ckpt
  test  loss:		0.36099
  test  Pearson's R:	0.80840+/-0.00000
  test  rsquare:	0.64453+/-0.00000
  test  slope:		1.02837+/-0.00000


In [10]:
for key in dataset.keys():
    print(str(key))

AARS_K562
AGGF1_HepG2
AGGF1_K562
AKAP8L_K562
AUH_HepG2
AUH_K562
BCCIP_HepG2
BUD13_HepG2
BUD13_K562
CDC40_HepG2
CPSF6_K562
CSTF2T_HepG2
CSTF2T_K562
CSTF2_HepG2
DDX24_K562
DDX3X_HepG2
DDX3X_K562
DDX42_K562
DDX55_HepG2
DDX55_K562
DDX59_HepG2
DDX6_HepG2
DDX6_K562
DGCR8_K562
DHX30_HepG2
DHX30_K562
DKC1_HepG2
DROSHA_K562
EFTUD2_HepG2
EFTUD2_K562
EIF3D_HepG2
EIF4G2_K562
EWSR1_K562
FAM120A_HepG2
FAM120A_K562
FASTKD2_HepG2
FASTKD2_K562
FKBP4_HepG2
FMR1_K562
FTO_K562
FUBP3_HepG2
FXR1_K562
FXR2_K562
GEMIN5_K562
GNL3_K562
GPKOW_K562
GRSF1_HepG2
GRWD1_HepG2
GTF2F1_HepG2
GTF2F1_K562
HLTF_K562
HNRNPA1_HepG2
HNRNPA1_K562
HNRNPC_HepG2
HNRNPK_HepG2
HNRNPK_K562
HNRNPM_HepG2
HNRNPM_K562
HNRNPUL1_HepG2
HNRNPUL1_K562
HNRNPU_HepG2
HNRNPU_K562
IGF2BP1_HepG2
IGF2BP1_K562
IGF2BP2_K562
IGF2BP3_HepG2
ILF3_HepG2
ILF3_K562
KHDRBS1_K562
KHSRP_K562
LARP4_HepG2
LARP4_K562
LARP7_HepG2
LARP7_K562
LIN28B_HepG2
LIN28B_K562
LSM11_HepG2
LSM11_K562
METAP2_K562
MTPAP_K562
NCBP2_HepG2
NCBP2_K562
NKRF_HepG2
NOL12_HepG2
NONO_K56

In [23]:
experiment = 'RBFOX2_HepG2'
#experiment = 'SRSF1_HepG2'
#experiment = 'QKI_HepG2'
#experiment = 'HNRNPC_HepG2'
data_path = '/media/peter/storage/eclip_data/dataset/eCLIP_strong_strong_nonspecific_100.h5'
dataset = h5py.File(data_path)
X_train = np.array(dataset['/'+experiment+'/X_train']).astype(np.float32)        
y_train = np.array(dataset['/'+experiment+'/Y_train']).astype(np.float32)
X_valid = np.array(dataset['/'+experiment+'/X_valid']).astype(np.float32)       
y_valid = np.array(dataset['/'+experiment+'/Y_valid']).astype(np.float32)
X_test = np.array(dataset['/'+experiment+'/X_test']).astype(np.float32)
y_test = np.array(dataset['/'+experiment+'/Y_test']).astype(np.float32)

# add another dimension to make a 4d tensor
X_train = np.expand_dims(X_train, axis=3).transpose([0, 2, 3, 1])
X_test = np.expand_dims(X_test, axis=3).transpose([0, 2, 3, 1])
X_valid = np.expand_dims(X_valid, axis=3).transpose([0, 2, 3, 1])

# dictionary for each dataset
clip_train = {'inputs': X_train, 'targets': y_train}
clip_valid = {'inputs': X_valid, 'targets': y_valid}
clip_test = {'inputs': X_test, 'targets': y_test}

In [24]:
X_train = clip_train['inputs']
y_train = clip_train['targets']

index = np.where(y_train[:,0]==1)[0]
X_pos = X_train[index]
y_pos = y_train[index]
index = np.where(y_train[:,0]==0)[0]
X_neg = X_train[index]
y_neg = y_train[index]


In [25]:
num_split = X_pos.shape[1] - 41

pos_score = []
for X in X_pos:
    X_split = []
    for i in range(num_split):
        X_split.append([X[i:i+41,:,:]])
    X_split = np.vstack(X_split)

    affinity = nntrainer.get_activations(sess, {'inputs': X_split})
    pos_score.append(np.max(affinity))
pos_score = np.array(pos_score)

neg_score = []
for X in X_neg:
    X_split = []
    for i in range(num_split):
        X_split.append([X[i:i+41,:,:]])
    X_split = np.vstack(X_split)

    affinity = nntrainer.get_activations(sess, {'inputs': X_split})
    neg_score.append(np.max(affinity))
neg_score = np.array(neg_score)

In [26]:
y_true = np.vstack([np.ones((len(pos_score),1)), np.zeros((len(neg_score),1))])
y_score = np.vstack([np.expand_dims(pos_score,axis=1), np.expand_dims(neg_score, axis=1)])

from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true, y_score)

roc_score = auc(fpr, tpr)
print(roc_score)


0.742437601984


In [11]:



def experiment_correpsondence(correspondence_path, experiment_names):
    import pandas as pd

    df = pd.read_csv(correspondence_path, header=None)
    rbp_names = df[2].as_matrix()
    rbp_indices = df[3].as_matrix()
    rncmpt_names = df[1].as_matrix()



    match = {}
    for j, rbp_name in enumerate(rbp_names):
        match[rbp_name] = []

    for j, rbp_name in enumerate(rbp_names):
        for i, name in enumerate(experiment_names):
            if rbp_name in name:
                match[rbp_name].append((rbp_indices[j], experiment_names[i], rncmpt_names[j]))

    clean_match = {}
    for key in match.keys():
        if match[key]:
            clean_match[key] = match[key]

    return clean_match

def load_clip_dataset(clip_path):
    dataset = h5py.File(clip_path)
    X_train = np.array(dataset['/'+experiment+'/X_train']).astype(np.float32)        
    y_train = np.array(dataset['/'+experiment+'/Y_train']).astype(np.float32)
    X_valid = np.array(dataset['/'+experiment+'/X_valid']).astype(np.float32)       
    y_valid = np.array(dataset['/'+experiment+'/Y_valid']).astype(np.float32)
    X_test = np.array(dataset['/'+experiment+'/X_test']).astype(np.float32)
    y_test = np.array(dataset['/'+experiment+'/Y_test']).astype(np.float32)

    # add another dimension to make a 4d tensor
    X_train = np.expand_dims(X_train, axis=3).transpose([0, 2, 3, 1])
    X_test = np.expand_dims(X_test, axis=3).transpose([0, 2, 3, 1])
    X_valid = np.expand_dims(X_valid, axis=3).transpose([0, 2, 3, 1])

    # dictionary for each dataset
    clip_train = {'inputs': X_train, 'targets': y_train}
    clip_valid = {'inputs': X_valid, 'targets': y_valid}
    clip_test = {'inputs': X_test, 'targets': y_test}

    return clip_train, clip_valid, clip_test

def binding_affinity_scores(sess, nntrainer, clip_train):

    X_train = clip_train['inputs']
    y_train = clip_train['targets']

    index = np.where(y_train[:,0]==1)[0]
    X_pos = X_train[index]
    y_pos = y_train[index]
    index = np.where(y_train[:,0]==0)[0]
    X_neg = X_train[index]
    y_neg = y_train[index]

    num_split = X_pos.shape[1] - 41

    pos_score = []
    for X in X_pos:
        X_split = []
        for i in range(num_split):
            X_split.append([X[i:i+41,:,:]])
        X_split = np.vstack(X_split)

        affinity = nntrainer.get_activations(sess, {'inputs': X_split})
        pos_score.append(np.max(affinity))
    pos_score = np.array(pos_score)

    neg_score = []
    for X in X_neg:
        X_split = []
        for i in range(num_split):
            X_split.append([X[i:i+41,:,:]])
        X_split = np.vstack(X_split)

        affinity = nntrainer.get_activations(sess, {'inputs': X_split})
        neg_score.append(np.max(affinity))
    neg_score = np.array(neg_score)

    y_true = np.vstack([np.ones((len(pos_score),1)), np.zeros((len(neg_score),1))])
    y_score = np.vstack([np.expand_dims(pos_score,axis=1), np.expand_dims(neg_score, axis=1)])

    return y_true, y_score


from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score


In [ ]:
models = ['deep_residual_model', 'conv_net']
normalize_method = 'log_norm' 
ss_type = 'seq'
best_path = '/home/peter/harvard/rbp/results/RNAcompete_2013/'+normalize_method+'_'+ss_type

clip_path = '/media/peter/storage/eclip_data/dataset/eCLIP_strong_strong_nonspecific_100.h5'
dataset = h5py.File(clip_path)
experiment_names = []
for key in dataset.keys():
    experiment_names.append(str(key))
    
correspondence_path = '../correspondences_eCLIP_RNACompete.csv'
match = experiment_correpsondence(correspondence_path, experiment_names)

data_path = '../data/RNAcompete_2013/rnac_SetAB_with_nan_RNAplfoldstructures.hdf5'
train_all, valid_all, test_all = load_data.RNAcompete_dataset(data_path, split_ratio=0.9, ss_type=ss_type)    

results = []
for key in match.keys():
    for experiments in match[key]:
        rbp_index = experiments[0]
        experiment = experiments[1]
        rcmpt_name = experiments[2]

        # process data 
        train, valid, test = helper.process_data(train_all, valid_all, test_all, rbp_index, method=normalize_method)

        for model in models:
            # import model
            if model == 'deep_residual_model':
                from models import deep_residual_model as genome_model
            elif model == 'conv_net':
                from models import conv_net as genome_model

            input_shape = list(train['inputs'].shape)
            input_shape[0] = None
            output_shape = train['targets'].shape
            model_layers, optimization = genome_model.model(input_shape, output_shape)

            # build neural network class
            nnmodel = nn.NeuralNet(seed=247)
            nnmodel.build_layers(model_layers, optimization)

            # compile neural trainer
            file_path = os.path.join(best_path, str(rbp_index)+'_'+model+'_'+ss_type)
            nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

            # initialize session
            sess = utils.initialize_session(nnmodel.placeholders)

            # load best parameters
            nntrainer.set_best_parameters(sess, verbose=0)

            loss, mean, std = nntrainer.test_model(sess, test, verbose=0)

            clip_train, clip_valid, clip_test = load_clip_dataset(clip_path)

            y_true, y_score = binding_affinity_scores(sess, nntrainer, clip_train)

            fpr, tpr, thresholds = roc_curve(y_true, y_score)
            roc_score = auc(fpr, tpr)

            results.append([rcmpt_name, experiment, model, roc_score, mean[0]])
            
            print("%s\t%s\tR=%0.4f\t%s\t%0.4f"%(rcmpt_name, experiment, mean[0],model, roc_score))
            

RNCMPT00020	FXR2_K562	R=0.4383	deep_residual_model	0.6079
RNCMPT00020	FXR2_K562	R=0.4301	conv_net	0.6111
RNCMPT00047	QKI_HepG2	R=0.0301	deep_residual_model	0.7631


In [23]:
results

[['RNCMPT00020',
  'FXR2_K562',
  'deep_residual_model',
  0.60788912267989526,
  0.43834302],
 ['RNCMPT00020', 'FXR2_K562', 'conv_net', 0.61105647214016312, 0.43006411],
 ['RNCMPT00047',
  'QKI_HepG2',
  'deep_residual_model',
  0.76311873507671757,
  0.030087756],
 ['RNCMPT00047', 'QKI_HepG2', 'conv_net', 0.81096601386601619, 0.032569751],
 ['RNCMPT00047',
  'QKI_K562',
  'deep_residual_model',
  0.82345037098642127,
  0.030087756],
 ['RNCMPT00047', 'QKI_K562', 'conv_net', 0.87599354217227554, 0.032569751],
 ['RNCMPT00022',
  'HNRNPA1_HepG2',
  'deep_residual_model',
  0.81437269668322387,
  0.85029775],
 ['RNCMPT00022', 'HNRNPA1_HepG2', 'conv_net', 0.79125941355551987, 0.82069033],
 ['RNCMPT00022',
  'HNRNPA1_K562',
  'deep_residual_model',
  0.86566108555142396,
  0.85029775],
 ['RNCMPT00022', 'HNRNPA1_K562', 'conv_net', 0.83112399599448195, 0.82069033],
 ['RNCMPT00106',
  'SRSF1_HepG2',
  'deep_residual_model',
  0.83136619255020228,
  0.74718547],
 ['RNCMPT00106', 'SRSF1_HepG2', 